# DS2 Assignment 1 - Nicolas Fernandez
## Predicting Property Prices from the Xindian District of New Taipei City, Taiwan
The task is to predict property prices using data taken from [UC Irvine](https://archive.ics.uci.edu/dataset/477/real+estate+valuation+data+set).

The precise dataset being used is a cleaned version uploaded within Janos Divenyi's [github repository](https://github.com/divenyijanos/ceu-ml/tree/2023/data/real_estate).

The task asks to create a 20% subsample from the data and then create a 70/30% split from that subsample for a training and test sets.

## Reading Data and Creating Training and Test Splits

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Setting the pseudo random seed for the due date of the assignment
prng = np.random.RandomState(20240322)

# Reading the data
#real_estate_data = pd.read_csv("https://raw.githubusercontent.com/divenyijanos/ceu-ml/2023/data/real_estate/real_estate.csv")
real_estate_data = pd.read_csv('real_estate.csv')
display(real_estate_data.head())
print(real_estate_data.info())

,id,transaction_date,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores,latitude,longitude,house_price_of_unit_area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414 entries, 0 to 413
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   414 non-null    int64  
 1   transaction_date                     414 non-null    float64
 2   house_age                            414 non-null    float64
 3   distance_to_the_nearest_MRT_station  414 non-null    float64
 4   number_of_convenience_stores         414 non-null    int64  
 5   latitude                             414 non-null    float64
 6   longitude                            414 non-null    float64
 7   house_price_of_unit_area             414 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 26.0 KB
None


From the data there's a superfluous `id` column that will be dropped. The `transaction_date` column contains information about the year and month of the observation however it is in a non-standard format (`pd.to_datetime()` cannot be used) and will not be included in initial features. The `latitude` and `longitude` columns contains geographical data for each column but that cannot be made sense of without futher information and therefore will not be included for initial feature modeling.

In [2]:
# Dropping the id column because it's unecessary
real_estate_data.drop(columns=['id'], inplace=True)

# Creating a randomly selected 20% sample of the data, creating target variable 'outcome'
real_estate_sample = real_estate_data.sample(frac=0.2)
outcome = real_estate_sample["house_price_of_unit_area"]

# Selecting numeric features as X variables, excluding target variable, transaction_date, latitude, and longitude
features = real_estate_sample.drop(columns=["house_price_of_unit_area", "transaction_date", "latitude", "longitude"])

# Creating training and testing splits from features and target, using a 30% split per specifications
X_train, X_test, y_train, y_test = train_test_split(features, outcome, test_size=0.3, random_state=prng)

print(f"Size of the training set: {X_train.shape}, size of the test set: {X_test.shape}")
X_train.head()

Size of the training set: (58, 3), size of the test set: (25, 3)


,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores
166,0.0,292.9978,6
91,9.1,1402.0160,0
289,13.9,289.3248,5
222,30.6,431.1114,10
147,3.2,489.8821,8


## Evalaution Function

In [3]:
# Defining the loss function, using Root Mean Squared Log Error
def calculateRMSLE(prediction, y_obs):
    return round(np.sqrt(np.mean((np.log(np.where(prediction < 0, 0, prediction) + 1) - np.log(y_obs + 1))**2)), 4)

RMSLE is an appropriate loss function since its a calculation that is less sensitive to outliers by design, appropriate for property price prediction. The risk (from a business perspective) from making a wrong prediction could be either under or over pricing homes in certain areas because values that the loss function may have treated as outliers may have been more important than they seemed.?

## Rigid Models
Initially OLS models without any non-linearity will be created

### Benchmark Model

In [4]:
# Creating a simple benchmark model for comparison, mean value of target variable for training set
benchmark = np.mean(y_train)

# Calculating performance of benchmark on test set using loss function
benchmark_pred = ['Benchmark', calculateRMSLE(benchmark, y_train), calculateRMSLE(benchmark, y_test)]

# Storing and displaying results in a dataframe
results = pd.DataFrame([benchmark_pred], columns=['Model', 'Train', 'Test'])
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148


### OLS Single Feature - `distance_to_the_nearest_MRT_station`

In [5]:
# Importing required library
from sklearn.linear_model import LinearRegression

# Creating OLS model trained on only `distance_to_the_nearest_MRT_station`
ols_single = LinearRegression().fit(X_train[['distance_to_the_nearest_MRT_station']], y_train)

# Creating predictions for model and calculating RMSLE
train_error = calculateRMSLE(ols_single.predict(X_train[['distance_to_the_nearest_MRT_station']]), y_train)
test_error = calculateRMSLE(ols_single.predict(X_test[['distance_to_the_nearest_MRT_station']]), y_test)
ols_single_pred = ['Single Feature OLS', train_error, test_error]

# Adding to results
results.loc[len(results)] = ols_single_pred
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148
1,Single Feature OLS,0.4312,0.3462


This model improves upon simple using the average of the target variable as a predictive model for our target as the RMSLE's improve however they are not vastly improved and this model only uses a single feature from the dataset. The model can be significantly improved as it likely does not catch much of the complexity of the data.

### Multivariate OLS
Building an OLS model that uses all the available meaningful features instead of a single variable.

In [6]:
# Creating multivariate OLS model and fitting to training data
ols_multi = LinearRegression().fit(X_train, y_train)

# Creating predictions
train_error = calculateRMSLE(ols_multi.predict(X_train), y_train)
test_error = calculateRMSLE(ols_multi.predict(X_test), y_test)
ols_multi_pred = ['Multi OLS', train_error, test_error]

# Adding to results
results.loc[len(results)] = ols_multi_pred
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148
1,Single Feature OLS,0.4312,0.3462
2,Multi OLS,0.2838,0.3302


The multivariate model using all the available meaningful features improved the predictive power on the test set, but not by much. This implies that using all the available features does increase the predictive power over using only a single feature, even if it's determined to be the most meaningful one, since it does not account for all facets of the data, however that the additional features don't really improve the model much.

## Flexible Models
Below more flexible models will be created

### Polynomial OLS with Interactions

In [7]:
# Importing required library
from sklearn.preprocessing import PolynomialFeatures

# Creating Polynomial object to create polynomial terms and interactions in training data
poly_interactions = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_poly = poly_interactions.fit_transform(X_train)

# Creating polynomial OLS model
ols_poly = LinearRegression().fit(X_poly, y_train)

# Creating predictions
train_error = calculateRMSLE(ols_poly.predict(X_poly), y_train)
test_error = calculateRMSLE(ols_poly.predict(poly_interactions.transform(X_test)), y_test)
ols_poly_pred = ['Poly OLS', train_error, test_error]

# Adding to results
results.loc[len(results)] = ols_poly_pred
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148
1,Single Feature OLS,0.4312,0.3462
2,Multi OLS,0.2838,0.3302
3,Poly OLS,0.3866,0.2809


# ADD ANALYSIS ABOUT THIS HERE AFTER FINAL RUN!!!

### Feature Engineering
In order to improve the model further feature engineering will need to be done. This means making sense of the transaction dates and geographical data.

From research, New Taipei City is actually within Taipei City and used to be known as Taipei County but was renamed because it exceeded a population of 2 million which meant that it had to be redifined as it's own city by law. Xindian is a county within the redefined Taipei County. For the purposes of this analysis the city center for Taipei City (not New Taipei) will be considered the city center for which all observations will be compared to. The geographical data will then be converted to km distance from the city center based on the provided coordinates and added to the dataframe as a new column. The latitude for the city center that will be used is 25.03583333 and the longitude is 121.5683333. The information was pulled from [here](https://gist.github.com/nkrusch/848e4d6ca0be879e2e03b78b051f17da) and verified with Google Maps.

The transaction dates within the data follow the format 2013.500, for example, with the number to the left of the decimal being the year and the number to the right of the decimal being the decimal representation of the month (.500 = June = <sup>6</sup>/<sub>12</sub>). The only quirk to this is that .000 = December. This data will be converted and split into two columns, one for the year and another for the month.

In [8]:
# Defining function for creating year and month columns from transaction_date as categorical columns
def date_fix(df):
    df['year'] = df['transaction_date'].astype(int)
    df['year'] = df['year'].astype('category')
    
    df['month'] = np.where(df['transaction_date'] % 1 == 0, 12, 
                           round((df['transaction_date'] % 1) * 12).astype(int))
    df['month'] = df['month'].astype('category')

# Defining function for calculating distance from city center in km
def cc_distance(lat, lon):
    # City center coordinates (Taipei)
    cc_lat = 25.03583333
    cc_lon = 121.5683333
    
    # Convert latitude and longitude from degrees to radians
    lat, lon, cc_lat, cc_lon = np.radians([lat, lon, cc_lat, cc_lon])

    # Calculating difference from city center
    diff_lon = lon - cc_lon
    diff_lat = lat - cc_lat

    # Using the Haversine Formula to calculate distance from radians
    a = np.sin(diff_lat / 2) ** 2 + np.cos(lat) * np.cos(cc_lat) * np.sin(diff_lon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    km_distance = 6371 * c  # 6371 = Radius of Earth in km

    return km_distance

# Running feature engineering functions on sample
date_fix(real_estate_sample)
real_estate_sample['km_distance_from_cc'] = real_estate_sample.apply(lambda row: cc_distance(row['latitude'], row['longitude']), axis=1)

# Creating feature engineered matrix and saving column names to a list
feature_matrix = real_estate_sample.drop(columns=['house_price_of_unit_area', 'transaction_date', 'latitude', 'longitude'])

# Creating new data split with feature engineered data with same specifications
X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(feature_matrix, outcome, test_size=0.3, random_state=prng)

print(f"Size of the FE training set: {X_train_fe.shape}, size of the FE test set: {X_test_fe.shape}")
X_train_fe.head()

Size of the FE training set: (58, 6), size of the FE test set: (25, 6)


,house_age,distance_to_the_nearest_MRT_station,number_of_convenience_stores,year,month,km_distance_from_cc
176,13.9,4573.7790,0,2012,10,12.184265
206,22.2,379.5575,10,2013,3,6.597912
79,18.0,1414.8370,1,2013,12,9.545572
347,17.4,6488.0210,1,2013,7,12.951991
234,8.0,2216.6120,4,2013,3,10.069093


### OLS Model with Feature Engineered Data

In [9]:
# Importing required libraries
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Setting up OneHotEncoder for handling categorical variables 'year' and 'month'
one_hot_encoder = OneHotEncoder(sparse_output=False, drop="first")
categorical_vars = ['year', 'month']

# Using ColumnTransformer to create the dummies based on defined OneHotEncoder
column_transformer = ColumnTransformer([("create_dummies", one_hot_encoder, categorical_vars)], 
                                       remainder="passthrough")

# Creating OLS model for FE data with Pipeline
pipe_ols_fe = Pipeline([("preprocess", column_transformer),
                   ("ols", LinearRegression())])

# Fitting the data
pipe_ols_fe.fit(X_train_fe, y_train_fe)

# Creating predictions
train_error = calculateRMSLE(pipe_ols_fe.predict(X_train_fe), y_train_fe)
test_error = calculateRMSLE(pipe_ols_fe.predict(X_test_fe), y_test_fe)
ols_fe_pred = ['FE OLS', train_error, test_error]

# Adding to results
results.loc[len(results)] = ols_fe_pred
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148
1,Single Feature OLS,0.4312,0.3462
2,Multi OLS,0.2838,0.3302
3,Poly OLS,0.3866,0.2809
4,FE OLS,0.2333,0.2286


The OLS model with feature engineering performed worse on the test set than the non-feature engieered OLS models. This could mean that the feature engineered variables are overfitting the model and resulting in the higher RMSLE on the test set, or that the feature engineered variables do not capture meaningful information and/or have introduced noise into the model.

### Polynomial OLS with Feature Engineered Data

In [10]:
# Importing required library
from sklearn.feature_selection import VarianceThreshold # To account for categorical values with zero variance

# Initiating VarianceThreshold to drop categorial variables with no variance
drop_no_variance = VarianceThreshold()

# Creating model with Pipeline
pipe_ols_poly_fe = Pipeline([
    ('preprocess', column_transformer),
    ('interactions', poly_interactions),
    ('drop_zero_variance', drop_no_variance),
    ('ols', LinearRegression())
])

# Fitting the data
pipe_ols_poly_fe.fit(X_train_fe, y_train_fe)

# Creating predictions
train_error = calculateRMSLE(pipe_ols_poly_fe.predict(X_train_fe), y_train_fe)
test_error = calculateRMSLE(pipe_ols_poly_fe.predict(X_test_fe), y_test_fe)
ols_poly_fe_pred = ['FE Poly OLS', train_error, test_error]

# Adding to results
results.loc[len(results)] = ols_poly_fe_pred
results

,Model,Train,Test
0,Benchmark,0.4329,0.4148
1,Single Feature OLS,0.4312,0.3462
2,Multi OLS,0.2838,0.3302
3,Poly OLS,0.3866,0.2809
4,FE OLS,0.2333,0.2286
5,FE Poly OLS,0.0605,1.3361


### Random Forest

In [9]:
# Importing required library
from sklearn.ensemble import RandomForestRegressor

# Creating a Random Forest model with Pipeline
pipe_rf = Pipeline([('preprocess', column_transformer),
    ("rf", RandomForestRegressor(random_state=prng))])

# Fitting the model to the feature engineered data
pipe_rf.fit(X_train_fe, y_train_fe)

# Creating predictions
train_error = calculateRMSLE(pipe_rf.predict(X_train_fe), y_train_fe)
test_error = calculateRMSLE(pipe_rf.predict(X_test_fe), y_test_fe)
rf_pred = ['RF', train_error, test_error]

# Adding to results
results.loc[len(results)] = rf_pred
results

,Model,Train,Test
0,Benchmark,0.4086,0.2715
1,Single Feature OLS,0.3264,0.2466
2,Multivariate OLS,0.2312,0.2798
3,FE OLS,0.2008,0.2212
4,RF,0.0800,0.1687


The Random Forest model significantly improves the RMSLE on both hte train and the test set.

In [10]:
# Creating dataframe for readability for displaying variable importance
rf_var_imp = pd.DataFrame(
    pipe_rf['rf'].feature_importances_, 
    pipe_rf[:-1].get_feature_names_out())\
    .reset_index()\
    .rename({"index": "variable", 0: "imp"}, axis=1)\
    .sort_values(by=["imp"], ascending=False)\
    .reset_index(drop = True)

# Creating cumulative sum column
rf_var_imp['cumulative_imp'] = rf_var_imp['imp'].cumsum()

# Displaying dataframe with formatting
rf_var_imp.style.format({'imp': lambda x: f'{x:,.1%}', 'cumulative_imp': lambda x: f'{x:,.1%}'})

,variable,imp,cumulative_imp
0,remainder__km_distance_from_cc,43.5%,43.5%
1,remainder__distance_to_the_nearest_MRT_station,29.7%,73.2%
2,remainder__house_age,14.0%,87.2%
3,remainder__number_of_convenience_stores,8.4%,95.5%
4,create_dummies__month_8,1.8%,97.3%
5,create_dummies__month_6,0.7%,97.9%
6,create_dummies__month_11,0.4%,98.3%
7,create_dummies__month_2,0.4%,98.7%
8,create_dummies__month_12,0.4%,99.0%
9,create_dummies__month_3,0.3%,99.3%
